# Creating FABnet IPv4 Network with Storage

## Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
           
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}") 

In [ ]:
slices = fablib.list_slices()


## (Optional): Query for Available Testbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [ ]:
try:
    fablib.list_sites()
except Exception as e:
    print(f"Exception: {e}")

# Create the Experiment Slice

NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

In [ ]:
server_site = 'DALL'

#client_sites = fablib.get_random_sites(count=5,avoid=[server_site, 'FIU','UCSD','GPN','CLEM','MAX','STAR'])
client_sites = fablib.get_random_sites(count=10,avoid=[server_site])




print(f"server_site: {server_site},  client_sites: {client_sites}")
slice_name = f'MySliceFABNETStorage2'




In [ ]:
try:
    print(f"Slice: {slice_name}")

    site = server_site
    storage_name = f'FABRIC_Staff_{site.lower()}_1T'
    node_name = f'{site}1'
    net_name=f"net{server_site}"
    
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    ifaces = []
    node = slice.add_node(name=node_name, site=site, cores=8, ram=32, disk=500)
    node.add_storage(name=storage_name)
    ifaces.append(node.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0])
    node.add_component(model='NVME_P4510', name='nvme')

    # Network
    net = slice.add_l3network(name=net_name, interfaces=ifaces, type='IPv4')

    #Submit Slice Request
    slice.submit(wait_interval=10)
except Exception as e:
    print(f"Exception: {e}")

## Configure the Server 

In [ ]:
try:
    site = server_site
    storage_name = f'FABRIC_Staff_{site.lower()}_1T'
    node_name = f'{site}1'
    net_name=f"net{server_site}"
    
    
    network = slice.get_network(name=net_name)
    network_available_ips = network.get_available_ips()
    network.show()
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network


try:
    node = slice.get_node(name=node_name)        
    node_iface = node.get_interface(network_name=net_name)  
    node_addr = network_available_ips.pop(0)
    server_addr = node_addr
    node_iface.ip_addr_add(addr=node_addr, subnet=network.get_subnet())
    output_file = f'logs/{node_name}.log'
    
    node.ip_route_add(subnet=IPv4Network('10.128.0.0/10'), gateway=network.get_gateway())
    
    stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}', quiet=True,  output_file=output_file)
    #print (stdout)
    
    stdout, stderr = node.execute(f'ip route list',quiet=True, output_file=output_file)
    #print (stdout)
except Exception as e:
    print(f"Exception: {e}")
    raise e

In [ ]:
try:
    node1 = slice.get_node(name=node_name)   
                                    
    rtnval = node1.upload_directory("fabric_node_tools",".")
    stdout, stderr = node1.execute(f'sudo chmod +x ./fabric_node_tools/*.sh; '
                                   f'sudo ./fabric_node_tools/host_tune_redhat.sh ', 
                                   output_file=output_file,
                                   quiet=True)

except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    site = server_site
    storage_name = f'FABRIC_Staff_{site.lower()}_1T'
    node_name = f'{site}1'
    net_name=f"net{server_site}"
    
    node1 = slice.get_node(name=node_name) 
    
    #print(f"{node}")
    storage = node.get_storage(storage_name)
    #print(f"{storage}")
    print(f"Storage Device Name: {storage.get_device_name()}")  

    #stdout, stderr = node1.execute(f'sudo dnf update -y ', quiet=False)
    #stdout, stderr = node1.execute(f'sudo dnf install -y epel-release ', quiet=False)

    
    stdout, stderr = node1.execute( f'sudo dnf install -y epel-release; '
                                    f'sudo /usr/bin/crb enable; '
                                    f'sudo dnf install -y httpd iproute-tc net-tools pciutils tcpdump iftop iperf3 vim; '
                                    f"sudo mount {storage.get_device_name()} /var/www/html; "
                                    f"sudo chmod -R 775 /var/www/html; "
                                    f'sudo systemctl start httpd; '
                                    f'sudo systemctl enable httpd; '
                                    f'sudo systemctl status httpd;'
                                    f'sudo setenforce 0',
                                    output_file=output_file,
                                    quiet=True)
    
    print('done!')
    
except Exception as e:
    print(f"Exception: {e}")

## Add the Client Nodes

In [ ]:
from itertools import zip_longest


try:
    print(f"Slice: {slice_name}")
    
    #Create Slice
    slice = fablib.get_slice(name=slice_name)

    site_batches = zip_longest(*(iter(client_sites),) * 5)
    for site_batch in site_batches:
        for site in site_batch:
            if site == None: 
                continue
            
            storage_name = f'FABRIC_Staff_{site.lower()}_1T'
            node_name = f'{site}1'
            net_name=f"net{site}"
            

            ifaces = []
            node = slice.add_node(name=node_name, site=site, cores=8, ram=32, disk=100)
            node.add_storage(name=storage_name)
            ifaces.append(node.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0])
            node.add_component(model='NVME_P4510', name='nvme')

            # Network
            net = slice.add_l3network(name=net_name, interfaces=ifaces, type='IPv4')

        #Submit Slice Request
        slice.submit(wait_interval=10)
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network



try:
    for site in client_sites:
        storage_name = f'FABRIC_Staff_{site.lower()}_1T'
        node_name = f'{site}1'
        net_name=f"net{site}"
        output_file = f'logs/{node_name}.log'

        network = slice.get_network(name=net_name)
        network_available_ips = network.get_available_ips()
        #network.show()
        
        
        
        node = slice.get_node(name=node_name) 
        node_iface = node.get_interface(network_name=net_name)  
        node_addr = network_available_ips.pop(0)
        node_iface.ip_addr_add(addr=node_addr, subnet=network.get_subnet())

        print(f"{node_name} {node_addr}")
        
        node.ip_route_add(subnet=IPv4Network('10.128.0.0/10'), gateway=network.get_gateway())

        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}', quiet=True, output_file=output_file)
        stdout, stderr = node.execute(f'ip route list', quiet=True, output_file=output_file)
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    
    threads = {}
    for site in client_sites:
        print(f"Starting site: {site}")
        storage_name = f'FABRIC_Staff_{site.lower()}_1T'
        node_name = f'{site}1'
        net_name=f"net{site}"
        output_file = f'logs/{node_name}.log'

    
        node = slice.get_node(name=node_name) 

        #print(f"{node}")
        storage = node.get_storage(storage_name)
        #print(f"{storage}")
        print(f"Storage Device Name: {storage.get_device_name()}")  


        rtnval = node.upload_directory("fabric_node_tools",".")
        stdout, stderr = node.execute(f'sudo chmod +x ./fabric_node_tools/*.sh; ',
                                       output_file=output_file)


        #stdout, stderr = node.execute(f'sudo dnf update -y ', quiet=False)
        #stdout, stderr = node.execute(f'sudo dnf install -y epel-release ', quiet=False)


        threads[site] = node.execute_thread(f'sudo dnf install -y epel-release; '
                                            f'sudo /usr/bin/crb enable; '
                                            f'sudo dnf install -y httpd iproute-tc net-tools pciutils tcpdump iftop iperf3 vim; '
                                            f'sudo ./fabric_node_tools/host_tune_redhat.sh; ' 
                                            f"sudo mkdir /mnt/fabric_storage; "
                                            f"sudo mkfs.ext4 {storage.get_device_name()}; "
                                            f"sudo mount {storage.get_device_name()} /mnt/fabric_storage; "
                                            f"sudo chmod -R 775 /mnt/fabric_storage; ",
                                            output_file=output_file)  
        

    for site,mythread in threads.items():
        print(f"site: {site}", end='')
        stdout,stderr = mythread.result()
        print('done!')
            
    print('all done!')
    
except Exception as e:
    print(f"Exception: {e}")
    raise e

## Run the Experiemnt

In [ ]:
try:
    #client_sites = ['MASS', 'TACC', 'UTAH', 'NCSA', 'WASH']
    #slice = fablib.get_slice('MySliceFABNETStorage')
    #server_addr = '10.133.129.2'

    
    threads = {}
    
    print(f"server_addr: {server_addr}")
    
    command =   f"sudo curl http://{server_addr}/file.1G --output /mnt/fabric_storage/file.1G; "
    command +=  f"sudo curl http://{server_addr}/file.10G --output /mnt/fabric_storage/file.10G; "
    command +=  f"sudo curl http://{server_addr}/file.20G --output /mnt/fabric_storage/file.20G; "
    command +=  f"sudo curl http://{server_addr}/file.50G --output /mnt/fabric_storage/file.50G; "
    command +=  f"sudo curl http://{server_addr}/file.100G --output /mnt/fabric_storage/file.100G; "
    
    print(f"command: {command}")
    
    for site in client_sites:
        storage_name = f'FABRIC_Staff_{site.lower()}_1T'
        node_name = f'{site}1'
        net_name=f"net{site}"
        output_file = f'logs/{node_name}.log'

    
        node = slice.get_node(name=node_name)  
        print(f"node: {node.get_name()}")
        threads[site] = node.execute_thread(command, output_file=output_file)  
        
    
    for site,mythread in threads.items():
        stdout,stderr = mythread.result()
        
        print(f"site: {site}, stdout: {stdout}, stderr: {stderr}")
        
except Exception as e:
    print(f"Exception: {e}")



In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    
    threads = {}
    
    #server_addr = '10.133.132.2'
    print(f"server_addr: {server_addr}")
    
    command =   f"sudo rm /mnt/fabric_storage/file.* ; "
    command +=  f"sudo sync; "
    command +=  f"sudo  sh -c 'echo 1 > /proc/sys/vm/drop_caches'; "


    for site in client_sites:
        print(f"site: {site}")
        storage_name = f'FABRIC_Staff_{site.lower()}_1T'
        node_name = f'{site}1'
        net_name=f"net{site}"
        output_file = f'logs/{node_name}.log'

    
        node1 = slice.get_node(name=node_name)                
        threads[site] = node1.execute_thread(command, output_file=output_file)  
        
    
    for site,mythread in threads.items():
        stdout,stderr = mythread.result()
        
        print(f"site: {site}, stdout: {stdout}, stderr: {stderr}")
        
except Exception as e:
    print(f"Exception: {e}")
    raise e



## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")